<a href="https://colab.research.google.com/github/FengruiJing/HIV_Prediction/blob/main/COVID_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 21 10:44:08 2023

@author: FJING
"""

import pandas as pd

df1 = pd.read_csv('D:/Data/COVID/Data/us-counties-2020.csv')
df2 = pd.read_csv('D:/Data/COVID/Data/us-counties-2021.csv')
df3 = pd.read_csv('D:/Data/COVID/Data/Population20172021.csv')
df12 = pd.concat([df1, df2], ignore_index=True)

# 移除包含缺失值或无穷大的行
df12 = df12.dropna(subset=['fips'])
df12 = df12[~df12['fips'].isin([float('inf'), float('-inf')])]

# 将县编号列从浮点数转换为整数
df12['fips'] = df12['fips'].astype(int)

#merge
df = df12.merge(df3[['fips', 'Population']], on='fips', how='left')


# 将日期列转换为日期格式
df['date'] = pd.to_datetime(df['date'])


# 按照县名和日期进行排序
df = df.sort_values(['fips', 'date'])


# 计算每天的新增病例数
df['new_cases'] = df.groupby('fips')['cases'].diff()


#
df['new_cases_rates'] = (df['new_cases'] / df['Population']) * 100000

df
# 对新增病例数进行七天滑动平均 右对齐滑动平均
df['7_day_avg'] = df.groupby('fips')['new_cases_rates'].rolling(window=7, center=False).mean().reset_index(level=0, drop=True)
df

# 添加表示每个日期所在的年份和周的列
df['year_week'] = df['date'].dt.isocalendar().year.astype(str) + '_' + df['date'].dt.isocalendar().week.astype(str)

111111
##筛选December 1, 2020–February 28, 2021 winter outbreak period
# 创建一个布尔掩码来筛选日期
mask = (df['date'] >= '2020-12-01') & (df['date'] <= '2021-02-28')

# 使用掩码来筛选数据
filtered_df = df.loc[mask]

## 计算每个县每周的累计7_day_avg
#df['weekly_total_7_day_avg'] = df.groupby(['fips', 'year_week'])['7_day_avg'].transform('sum')

# 计算每个县每周的累计7_day_avg
weekly_df = filtered_df.groupby(['fips', 'year_week'])['7_day_avg'].agg('sum').reset_index().rename(columns={'7_day_avg': 'weekly_total_7_day_avg'})

# 创建一个布尔掩码，对于每个元素，如果它是﹣数，返回True，否则返回False
mask0 = weekly_df['weekly_total_7_day_avg'] < 0

weekly_df = weekly_df[~mask0]

# 将数据保存为CSV文件
weekly_df.to_csv('D:/Data/COVID/Data/WinterweeklyCOVID.csv', index=False)

# 获取所有唯一的年-周标识符
unique_year_weeks = weekly_df['year_week'].unique()

# 对每个唯一的年-周标识符
for year_week in unique_year_weeks:
    # 选择该周的数据
    week_data = weekly_df[weekly_df['year_week'] == year_week]

    # 保存为CSV文件，文件名包含年-周标识符
    week_data.to_csv(f'D:/Data/COVID/Data/{year_week}_COVID.csv', index=False)

111111
##筛选July 15–October 31,2021 (SARS-CoV-2 B.1.617.2 [Delta] predominance)
# 创建一个布尔掩码来筛选日期2
mask2 = (df['date'] >= '2021-07-15') & (df['date'] <= '2021-10-31')

# 使用掩码来筛选数据
filtered_df2 = df.loc[mask2]

# 计算每个县每周的累计7_day_avg
weekly_df2 = filtered_df2.groupby(['fips', 'year_week'])['7_day_avg'].agg('sum').reset_index().rename(columns={'7_day_avg': 'weekly_total_7_day_avg'})


# 创建一个布尔掩码，对于每个元素，如果它是﹣数，返回True，否则返回False
mask00 = weekly_df2['weekly_total_7_day_avg'] < 0

weekly_df2 = weekly_df2[~mask00]

# 将数据保存为CSV文件
weekly_df2.to_csv('D:/Data/COVID/Data/DeltaweeklyCOVID.csv', index=False)


# 获取所有唯一的年-周标识符
unique_year_weeks = weekly_df2['year_week'].unique()

# 对每个唯一的年-周标识符
for year_week in unique_year_weeks:
    # 选择该周的数据
    week_data2 = weekly_df2[weekly_df2['year_week'] == year_week]

    # 保存为CSV文件，文件名包含年-周标识符
    week_data2.to_csv(f'D:/Data/COVID/Data/{year_week}_COVID.csv', index=False)


111111
##筛选'2021-03-01 - 2021-07-14 Sprin period

# 创建一个布尔掩码来筛选日期2
mask2 = (df['date'] >= '2021-03-01') & (df['date'] <= '2021-07-14')


# 使用掩码来筛选数据
filtered_df2 = df.loc[mask2]

# 计算每个县每周的累计7_day_avg
weekly_df2 = filtered_df2.groupby(['fips', 'year_week'])['7_day_avg'].agg('sum').reset_index().rename(columns={'7_day_avg': 'weekly_total_7_day_avg'})


# 创建一个布尔掩码，对于每个元素，如果它是﹣数，返回True，否则返回False
mask00 = weekly_df2['weekly_total_7_day_avg'] < 0

weekly_df2 = weekly_df2[~mask00]

# 将数据保存为CSV文件
weekly_df2.to_csv('D:/Data/COVID/Data/SpringweeklyCOVID.csv', index=False)


1111111111111111111
1111111111111111111
####2023-06-21 calculate ten-year connection indicies


import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/cb_2019_us_county_20mProjected1.txt',delimiter='\t')

# split the DataFrame into two separate DataFrames
df21 = df2.iloc[:len(df2)//2, :]
df22 = df2.iloc[len(df2)//2:, :]
# save the two sub-files as separate text files
df21.to_csv('D:/Data/COVID/Data/textfile_part1.txt', sep='\t', index=False)
df22.to_csv('D:/Data/COVID/Data/textfile_part2.txt', sep='\t', index=False)

#combine two csv files
df221 = pd.read_csv('D:/Data/COVID/Data/textfile_part1.csv')
df222 = pd.read_csv('D:/Data/COVID/Data/textfile_part2.csv')
combined_df = pd.concat([df221, df222], ignore_index=True)
# save the concatenated DataFrame as a single CSV file
combined_df.to_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv', index=False)

combined_df.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
combined_df.rename(columns={'a': 'user_loc'}, inplace=True)

# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(combined_df,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2020_49_COVIDWeightedDIS.csv', index=True)

111111-50
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2020_50_COVIDWeightedDIS.csv', index=True)


111111-51
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2020_51_COVIDWeightedDIS.csv', index=True)



111111-52
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2020_52_COVIDWeightedDIS.csv', index=True)




111111-53
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2020_53_COVIDWeightedDIS.csv', index=True)


111111-2021-1
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_1_COVIDWeightedDIS.csv', index=True)




111111-2021-2
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_2_COVIDWeightedDIS.csv', index=True)


111111-2021-3
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_3_COVIDWeightedDIS.csv', index=True)

111111-2021-4
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_4_COVIDWeightedDIS.csv', index=True)

111111-2021-5
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_5_COVIDWeightedDIS.csv', index=True)

111111-2021-6
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_6_COVIDWeightedDIS.csv', index=True)

111111-2021-7
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_7_COVIDWeightedDIS.csv', index=True)


111111-2021-8
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Winterdata/2021_8_COVIDWeightedDIS.csv', index=True)



import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
print(df)

#pivolt table step
pivot_table=df.pivot_table(values='scaled_sci', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/totalSCIeachCounty.csv', index=True)


#PCI
df = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')

#pivolt table step
pivot_table=df.pivot_table(values='pci', index='place_i', aggfunc='sum')
#save the result
pivot_table.to_csv('D:/Data/COVID/Data/totalPCIeachCounty.csv', index=True)

111111111111111111111
111111111111111111111

#combine all docs to a new doc
#PCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/extractPCI'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df = combined_df[['user_loc', 'year_week', 'W1_PCIHIV']]

# 查看结果
print(combined_df)


#SCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/extractSCI'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df1 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df1 = combined_df1[['user_loc', 'year_week', 'W1_SCIHIV']]

# 查看结果
print(combined_df1)

#SCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/extractDIS'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df2 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df2 = combined_df2[['user_loc', 'year_week', 'W1_DistanceHIV']]

# 查看结果
print(combined_df2)

#COVID
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/extractCOVID'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df3 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df3 = combined_df3[['fips', 'year_week', 'weekly_total_7_day_avg']]
combined_df3.rename(columns={'fips': 'user_loc'}, inplace=True)

# 查看结果
print(combined_df3)



# 将4个数据集合并为一个
merged_df = combined_df.merge(combined_df1, on=['user_loc', 'year_week'], how='outer')
merged_df = merged_df.merge(combined_df2, on=['user_loc', 'year_week'], how='outer')
merged_df = merged_df.merge(combined_df3, on=['user_loc', 'year_week'], how='outer')

# 显示合并后的数据集
print(merged_df)


#SCI
sci_df = pd.read_csv('D:/Data/COVID/Data/totalSCIeachCounty.csv')
final_df = merged_df.merge(sci_df, on='user_loc', how='outer')

#PCI
pci_df = pd.read_csv('D:/Data/COVID/Data/totalPCIeachCounty.csv')
pci_df.rename(columns={'place_i': 'user_loc'}, inplace=True)
final_df = final_df.merge(pci_df, on='user_loc', how='outer')

111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_pcicov'] = np.log(final_df['W1_PCIHIV'])
final_df['log_scicov'] = np.log(final_df['W1_SCIHIV'])
final_df['log_discov'] = np.log(final_df['W1_DistanceHIV'])
final_df['log_incidence'] = np.log(final_df['weekly_total_7_day_avg'])
final_df['log_pci'] = np.log(final_df['pci'])
final_df['log_sci'] = np.log(final_df['scaled_sci'])

# 显示DataFrame以检查结果
print(final_df)
final_df.columns
final_df.rename(columns={'user_loc': 'fips'}, inplace=True)
final_df.rename(columns={'W1_PCIHIV': 'pcicov'}, inplace=True)
final_df.rename(columns={'W1_SCIHIV': 'scicov'}, inplace=True)
final_df.rename(columns={'W1_DistanceHIV': 'discov'}, inplace=True)

# 删除包含NaN值的行
final_df = final_df.dropna()


111111111111111#确定每个县是否有13个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于13的县的列表
counties_to_remove = county_counts[county_counts != 13].index

# 只保留数据量等于13的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


#rural-urban
urbanrural = pd.read_csv('D:/Data/COVID/Data/UrbanRural.csv')
urbanrural.rename(columns={'FIPS': 'fips'}, inplace=True)
final_df = final_df.merge(urbanrural, on='fips', how='outer')
# 删除包含NaN值的行
final_df = final_df.dropna()
final_df.to_csv('D:/Data/COVID/Data/FinalUSdata.csv', index=False)

11111111111111111111
11111111111111111111#select SouthCarolinaData
# 创建一个布尔掩码来筛选fips编号
mask = (final_df['fips'] >= 45001) & (final_df['fips'] <= 45091)

# 使用掩码来筛选数据
filtered_df = final_df.loc[mask]

# 显示DataFrame以检查结果
print(filtered_df)
# 按照 'fips' 和 'year_week' 进行排序
sorted_df1 = filtered_df.sort_values(['fips', 'year_week'])

# 保存到 CSV 文件
sorted_df1.to_csv('D:/Data/COVID/Data/FinalSCdata.csv', index=False)

11111111111111111111
11111111111111111111#select MetropolitanCoreArea
# 创建一个布尔掩码来筛选fips编号
mask = (final_df['RUCC_2013']==1)

# 使用掩码来筛选数据
filtered_df = final_df.loc[mask]

# 显示DataFrame以检查结果
print(filtered_df)
# 按照 'fips' 和 'year_week' 进行排序
sorted_df2 = filtered_df.sort_values(['fips', 'year_week'])

# 保存到 CSV 文件
sorted_df2.to_csv('D:/Data/COVID/Data/FinaldataCoreMetro.csv', index=False)

1111111111111111
111111111111111111111
1111111111111111111111
###total dataset
final_df = pd.read_csv('D:/Data/COVID/Data/FinalUSdata.csv')
# 删除包含NaN值的行
final_df.columns
final_df = final_df.dropna()
final_df = final_df[final_df['weekly_total_7_day_avg'] != 0]

111111111111111#确定每个县是否有13个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于13的县的列表
counties_to_remove = county_counts[county_counts != 13].index

# 只保留数据量等于13的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]

# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/FinaldataUSforRNN.csv', index=False)


####test
#####COVID SC
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv('D:/Data/COVID/Data/FinaldataCoreMetro.csv')
df.columns

df['fips'] = df['fips'].astype('category')
df['year_week'] = df['year_week'].astype(str)

all_predictions = []
all_maes = []
all_mapes = []

for fips in df['fips'].unique():
    print(f'Processing fips {fips}')

    county_df = df[df['fips'] == fips]

    train_df = county_df[(county_df['year_week'] >= '2020_49') & (county_df['year_week'] < '2021_8')]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['log_incidence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year_week in ['2021_5', '2021_6', '2021_7', '2021_8']:
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0].item()

            predictions.append((fips, year_week, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year_week in train_df['year_week'].values:
                next_year_data = train_df.loc[train_df['year_week'] == year_week]['log_incidence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['year_week'] >= '2021_5') & (county_df['year_week'] <= '2021_8')]['log_incidence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for fips {fips} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['fips', 'year_week', 'log_incidence'])
predictions_df.to_csv('D:/Data/COVID/Data/FinaldataCoreMetroPredictionBestHIVlog.csv', index=False)


####test
#####WHOLEUS
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNN.csv')
df.columns

df['fips'] = df['fips'].astype('category')
df['year_week'] = df['year_week'].astype(str)

all_predictions = []
all_maes = []
all_mapes = []

for fips in df['fips'].unique():
    print(f'Processing fips {fips}')

    county_df = df[df['fips'] == fips]

    train_df = county_df[(county_df['year_week'] >= '2020_49') & (county_df['year_week'] < '2021_8')]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['log_incidence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year_week in ['2021_5', '2021_6', '2021_7', '2021_8']:
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0].item()

            predictions.append((fips, year_week, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year_week in train_df['year_week'].values:
                next_year_data = train_df.loc[train_df['year_week'] == year_week]['log_incidence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['year_week'] >= '2021_5') & (county_df['year_week'] <= '2021_8')]['log_incidence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for fips {fips} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['fips', 'year_week', 'log_incidence'])
predictions_df.to_csv('D:/Data/COVID/Data/ResultsofWholeUS/USBestCOVlog.csv', index=False)



11111111111111111
1111111111111111111
1111111111111111111111
###Spring period calculation

import pandas as pd
# 将数据保存为CSV文件
df = pd.read_csv('D:/Data/COVID/Data/SpringweeklyCOVID.csv')


#SCI
sci_df = pd.read_csv('D:/Data/COVID/Data/totalSCIeachCounty.csv')
sci_df.rename(columns={'user_loc': 'fips'}, inplace=True)

final_df = df.merge(sci_df, on='fips', how='outer')

#PCI
pci_df = pd.read_csv('D:/Data/COVID/Data/totalPCIeachCounty.csv')
pci_df.rename(columns={'place_i': 'fips'}, inplace=True)
final_df = final_df.merge(pci_df, on='fips', how='outer')
final_df.columns

#urbanrural
UrbanRural_df = pd.read_csv('D:/Data/COVID/Data/UrbanRural.csv')
UrbanRural_df.rename(columns={'FIPS': 'fips'}, inplace=True)
final_df = final_df.merge(UrbanRural_df, on='fips', how='outer')

final_df = final_df.dropna()


111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_incidence'] = np.log(final_df['weekly_total_7_day_avg'])
final_df['log_pci'] = np.log(final_df['pci'])
final_df['log_sci'] = np.log(final_df['scaled_sci'])

# 按照 'fips' 和 'year_week' 进行排序
final_df = final_df.sort_values(['fips', 'year_week'])

# no zero values
final_df = final_df.loc[final_df['weekly_total_7_day_avg'] != 0]
# no zero values
final_df = final_df.loc[final_df['log_incidence'] != 0]


111111111111111#确定每个县是否有20个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于12的县的列表
counties_to_remove = county_counts[county_counts != 20].index

# 只保留数据量等于12的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNSpring.csv', index=False)

111111
# 创建新列 'year' 和 'week'，并将它们转换为整数，这是因为你的 year_week 列被视为字符串，而不是数字。在字符串排序中，'2021_9' 实际上会出现在 '2021_10' 之后，因为字符串的排序是从左到右，字符接字符地进行的。
df = pd.read_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNSpring.csv')
# 创建新列 'year' 和 'week'，并将它们转换为整数
df[['year', 'week']] = df['year_week'].str.split('_', expand=True).astype(int)

# 然后按照 'year' 和 'week' 列进行排序
df.sort_values(['fips', 'year', 'week'], inplace=True)
df.to_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNSpring.csv', index=False)


111111111111111111111111111
1111111111111111111111
###Delta period calculation
import pandas as pd

df = pd.read_csv('D:/Data/COVID/Data/DeltaweeklyCOVID.csv')


#SCI
sci_df = pd.read_csv('D:/Data/COVID/Data/totalSCIeachCounty.csv')
sci_df.rename(columns={'user_loc': 'fips'}, inplace=True)

final_df = df.merge(sci_df, on='fips', how='outer')

#PCI
pci_df = pd.read_csv('D:/Data/COVID/Data/totalPCIeachCounty.csv')
pci_df.rename(columns={'place_i': 'fips'}, inplace=True)
final_df = final_df.merge(pci_df, on='fips', how='outer')
final_df.columns

#urbanrural
UrbanRural_df = pd.read_csv('D:/Data/COVID/Data/UrbanRural.csv')
UrbanRural_df.rename(columns={'FIPS': 'fips'}, inplace=True)
final_df = final_df.merge(UrbanRural_df, on='fips', how='outer')

final_df = final_df.dropna()


111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_incidence'] = np.log(final_df['weekly_total_7_day_avg'])
final_df['log_pci'] = np.log(final_df['pci'])
final_df['log_sci'] = np.log(final_df['scaled_sci'])

# 按照 'fips' 和 'year_week' 进行排序
final_df = final_df.sort_values(['fips', 'year_week'])

# no zero values
final_df = final_df.loc[final_df['weekly_total_7_day_avg'] != 0]
# no zero values
final_df = final_df.loc[final_df['log_incidence'] != 0]


111111111111111#确定每个县是否有20个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于12的县的列表
counties_to_remove = county_counts[county_counts != 16].index

# 只保留数据量等于16的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNDelta.csv', index=False)


1111111111111
11111111111111
#HIV+PCI+SCI totaldataset COVID Delta
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNDelta.csv')

df['fips'] = df['fips'].astype('category')
df['year_week'] = df['year_week'].astype(str)

all_predictions = []
all_maes = []
all_mapes = []

for fips in df['fips'].unique():
    print(f'Processing fips {fips}')

    county_df = df[df['fips'] == fips]

    train_df = county_df[(county_df['year_week'] >= '2021_28') & (county_df['year_week'] < '2021_43')]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['log_incidence', 'log_sci','log_pci']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year_week in ['2021_37', '2021_38', '2021_39', '2021_40','2021_41', '2021_42', '2021_43']:
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0].item()
            # Save prediction instead of replacing with true value
            predictions.append((fips, year_week, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year_week in train_df['year_week'].values:
                next_year_data = train_df.loc[train_df['year_week'] == year_week][['log_incidence', 'log_sci','log_pci']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['year_week'] >= '2021_37') & (county_df['year_week'] <= '2021_43')]['log_incidence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for fips {fips} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['fips', 'year_week', 'Predicted_HIV_Rate'])
predictions_df.to_csv('D:/Data/COVID/Data/USBestCOVlogTotalPCIlogTotalSCIlogDelta.csv', index=False)



111111111111111111111111111
1111111111111111111111
###Check use metro data and population density evaluation
import pandas as pd

df = pd.read_csv('D:/Data/COVID/Data/FinaldataCoreMetro.csv')


#PopulationDensity
Pop_df = pd.read_csv('D:/Data/COVID/Data/PopulationDensity.csv')
Pop_df.rename(columns={'FIPS': 'fips'}, inplace=True)

final_df = df.merge(Pop_df, on='fips', how='outer')


111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_PopulationDensity'] = np.log(final_df['PopulationDensity'])


# 按照 'fips' 和 'year_week' 进行排序
final_df = final_df.sort_values(['fips', 'year_week'])

# no zero values
final_df = final_df.loc[final_df['log_PopulationDensity'] != 0]
# no zero values
final_df = final_df.loc[final_df['year_week'] != 0]

final_df.dropna(subset=['year_week'], inplace=True)


111111111111111#确定每个县是否有20个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于12的县的列表
counties_to_remove = county_counts[county_counts != 13].index

# 只保留数据量等于12的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/FinaldataCoreMetroAddPop.csv', index=False)


#RNN test
###2023-06-30
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv('D:/Data/COVID/Data/FinaldataCoreMetroAddPop.csv')

df['fips'] = df['fips'].astype('category')
df['year_week'] = df['year_week'].astype(str)

all_predictions = []
all_maes = []
all_mapes = []

for fips in df['fips'].unique():
    print(f'Processing fips {fips}')

    county_df = df[df['fips'] == fips]

    train_df = county_df[(county_df['year_week'] >= '2020_49') & (county_df['year_week'] < '2021_8')]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['log_incidence', 'log_PopulationDensity']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year_week in ['2021_5', '2021_6', '2021_7', '2021_8']:
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0].item()
            # Save prediction instead of replacing with true value
            predictions.append((fips, year_week, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year_week in train_df['year_week'].values:
                next_year_data = train_df.loc[train_df['year_week'] == year_week][['log_incidence', 'log_PopulationDensity']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['year_week'] >= '2021_5') & (county_df['year_week'] <= '2021_8')]['log_incidence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for fips {fips} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['fips', 'year_week', 'log_incidence'])
predictions_df.to_csv('D:/Data/COVID/Data/MetroCoreResults/USBestCOVloglog_PopulationDensity.csv', index=False)


111111111111111111111111111
1111111111111111111111
###Check use delta  data and population density evaluation
import pandas as pd

df = pd.read_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNDelta.csv')


#PopulationDensity
Pop_df = pd.read_csv('D:/Data/COVID/Data/PopulationDensity.csv')
Pop_df.rename(columns={'FIPS': 'fips'}, inplace=True)

final_df = df.merge(Pop_df, on='fips', how='outer')


111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_PopulationDensity'] = np.log(final_df['PopulationDensity'])


# 按照 'fips' 和 'year_week' 进行排序
final_df = final_df.sort_values(['fips', 'year_week'])

# no zero values
final_df = final_df.loc[final_df['log_PopulationDensity'] != 0]
# no zero values
final_df = final_df.loc[final_df['year_week'] != 0]

final_df.dropna(subset=['year_week'], inplace=True)


111111111111111#确定每个县是否有20个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于16的县的列表
counties_to_remove = county_counts[county_counts != 16].index

# 只保留数据量等于12的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNDeltaPopulation.csv', index=False)



111111111111111111111111111
1111111111111111111111
###Check use winter data and population density evaluation
import pandas as pd

df = pd.read_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNN.csv')


#PopulationDensity
Pop_df = pd.read_csv('D:/Data/COVID/Data/PopulationDensity.csv')
Pop_df.rename(columns={'FIPS': 'fips'}, inplace=True)

final_df = df.merge(Pop_df, on='fips', how='outer')


111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_PopulationDensity'] = np.log(final_df['PopulationDensity'])


# 按照 'fips' 和 'year_week' 进行排序
final_df = final_df.sort_values(['fips', 'year_week'])

# no zero values
final_df = final_df.loc[final_df['log_PopulationDensity'] != 0]
# no zero values
final_df = final_df.loc[final_df['year_week'] != 0]

final_df.dropna(subset=['year_week'], inplace=True)


111111111111111#确定每个县是否有20个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于16的县的列表
counties_to_remove = county_counts[county_counts != 13].index

# 只保留数据量等于12的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNNPopulation.csv', index=False)
final_df.columns


df = pd.read_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNN.csv')
df.columns

final_df=df.dropna(subset=['log_pci'], inplace=True)


#COVIDPCINEW
###2023-07-03
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv('D:/Data/COVID/Data/FinaldataCoreMetroAddPop.csv')

df['fips'] = df['fips'].astype('category')
df['year_week'] = df['year_week'].astype(str)

all_predictions = []
all_maes = []
all_mapes = []

for fips in df['fips'].unique():
    print(f'Processing fips {fips}')

    county_df = df[df['fips'] == fips]

    train_df = county_df[(county_df['year_week'] >= '2020_49') & (county_df['year_week'] < '2021_8')]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['log_pcicov', 'log_incidence']])

    x_train = np.array([train[i-9:i, 0] for i in range(9, len(train))])
    y_train = np.array([train[i, 1] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Get the future log_pcicov values for the prediction periods
        future_log_pcicov = county_df[county_df['year_week'].isin(['2021_5', '2021_6', '2021_7', '2021_8'])]['log_pcicov'].values

        # Initialize an array to hold the prediction inputs
        future_inputs = np.empty((0, 9, 1))

        # For each prediction period, get the last 9 years of log_pcicov values, including the future ones, and add it to future_inputs
        for i in range(4):
            last_9_years = np.hstack((train[-(9 - i):, 0], future_log_pcicov[:i+1]))
            last_9_years = last_9_years.reshape((1, 9, 1))
            future_inputs = np.vstack((future_inputs, last_9_years))

        # Use the future log_pcicov values as input to make the predictions
        predictions = model.predict(future_inputs)

        # Transform the predictions back to the original scale
        predictions_transformed_back = [scaler.inverse_transform(np.hstack((future_inputs[i], predictions[i].reshape(-1, 1))))[-1][1] for i in range(4)]

        # Prepare the predictions list
        predictions = [(fips, year_week, prediction) for year_week, prediction in zip(['2021_5', '2021_6', '2021_7', '2021_8'], predictions_transformed_back)]

        true_values = county_df[(county_df['year_week'] >= '2021_5') & (county_df['year_week'] <= '2021_8')]['log_incidence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values)
        mape = mean_absolute_percentage_error(true_values, predicted_values)

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for fips {fips} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['fips', 'year_week', 'log_incidence'])
predictions_df.to_csv('D:/Data/COVID/Data/MetroCoreResults/USBestCOVIDPCINew.csv', index=False)



1111111111111111111
1111111111111111111
####2023-06-21 calculate ten-year connection indicies

import pandas as pd
weekly_df = pd.read_csv('D:/Data/COVID/Data/FinaldataHIVdiagnoseUSforRNNDelta.csv')
unique_year_weeks = weekly_df['year_week'].unique()
# 对每个唯一的年-周标识符
for year_week in unique_year_weeks:
    # 选择该周的数据
    week_data = weekly_df[weekly_df['year_week'] == year_week]

    # 保存为CSV文件，文件名包含年-周标识符
    week_data.to_csv(f'D:/Data/COVID/Data/Delta/{year_week}_COVID.csv', index=False)


# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join.columns


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci_x'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci_x'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/cb_2019_us_county_20mProjected1.txt',delimiter='\t')

# split the DataFrame into two separate DataFrames
df21 = df2.iloc[:len(df2)//2, :]
df22 = df2.iloc[len(df2)//2:, :]
# save the two sub-files as separate text files
df21.to_csv('D:/Data/COVID/Data/textfile_part1.txt', sep='\t', index=False)
df22.to_csv('D:/Data/COVID/Data/textfile_part2.txt', sep='\t', index=False)

#combine two csv files
df221 = pd.read_csv('D:/Data/COVID/Data/textfile_part1.csv')
df222 = pd.read_csv('D:/Data/COVID/Data/textfile_part2.csv')
combined_df = pd.concat([df221, df222], ignore_index=True)
# save the concatenated DataFrame as a single CSV file
combined_df.to_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv', index=False)

combined_df.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
combined_df.rename(columns={'a': 'user_loc'}, inplace=True)

# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(combined_df,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedDIS.csv', index=True)


111111-29
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_29_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_29_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_29_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_29_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_30_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_29_COVIDWeightedDIS.csv', index=True)


111111-30
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_30_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_30_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_30_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_30_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_30_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_30_COVIDWeightedDIS.csv', index=True)


111111-31
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_31_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_31_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_31_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_31_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_31_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_31_COVIDWeightedDIS.csv', index=True)

111111-32
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_32_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_32_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_32_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_32_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_32_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_32_COVIDWeightedDIS.csv', index=True)

111111-33
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_33_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_33_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_33_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_33_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_33_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_33_COVIDWeightedDIS.csv', index=True)

111111-34
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_34_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_34_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_34_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_34_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_34_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_34_COVIDWeightedDIS.csv', index=True)

111111-35
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_35_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_35_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_35_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_35_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_35_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_35_COVIDWeightedDIS.csv', index=True)

111111-36
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_36_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_36_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_36_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_36_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_36_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_36_COVIDWeightedDIS.csv', index=True)

111111-37
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_37_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_37_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_37_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_37_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_37_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_37_COVIDWeightedDIS.csv', index=True)

111111-38
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_38_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_38_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_38_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_38_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_38_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_38_COVIDWeightedDIS.csv', index=True)

111111-39
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_39_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_39_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_39_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_39_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_39_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_39_COVIDWeightedDIS.csv', index=True)

111111-40
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_40_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_40_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_40_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_40_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_40_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_40_COVIDWeightedDIS.csv', index=True)

111111-41
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_41_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_41_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_41_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_41_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_41_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_41_COVIDWeightedDIS.csv', index=True)

111111-42
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_42_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_42_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_42_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_42_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_42_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_42_COVIDWeightedDIS.csv', index=True)

111111-43
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_43_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_43_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_43_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_43_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_43_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_43_COVIDWeightedDIS.csv', index=True)

111111-1
import pandas as pd

# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


print(df)
# Read HIV file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)


# merge two dataframes

inner_join = pd.merge(df,
					df1,
					on ='fips',
					how ='inner')
inner_join


# multiply two columns
inner_join['W1_SCIHIV']= inner_join['scaled_sci'] * inner_join['weekly_total_7_day_avg']

#show last20 rows
last_20_rows = inner_join.tail(20)
print(last_20_rows)

#pivolt table step
pivot_table=inner_join.pivot_table(values='W1_SCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedSCI.csv', index=True)


#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)
print(df2)
# Read HIV 2018 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)

# merge two dataframes

inner_join1 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join1


# multiply two columns
inner_join1['W1_PCIHIV']= inner_join1['pci'] * inner_join1['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table1=inner_join1.pivot_table(values='W1_PCIHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table1.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedPCI.csv', index=True)


#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)
# Read HIV 201814 file into DataFrame
df1 = pd.read_csv('D:/Data/COVID/Data/Delta/2021_28_COVID.csv')
print(df1)

# merge two dataframes

inner_join2 = pd.merge(df2,
					df1,
					on ='fips',
					how ='inner')
inner_join2


# multiply two columns
inner_join2['W1_DistanceHIV']= inner_join2['GEOID'] * inner_join2['weekly_total_7_day_avg']

#pivolt table step add all together
pivot_table2=inner_join2.pivot_table(values='W1_DistanceHIV', index='user_loc', aggfunc='sum')

#save the result
pivot_table2.to_csv('D:/Data/COVID/Data/Delta/2021_28_COVIDWeightedDIS.csv', index=True)



111111111111111111111
111111111111111111111

#combine all docs to a new doc
#PCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/Delta/extractPCI'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df = combined_df[['user_loc', 'year_week', 'W1_PCIHIV']]

# 查看结果
print(combined_df)


#SCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/Delta/extractSCI'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df1 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df1 = combined_df1[['user_loc', 'year_week', 'W1_SCIHIV']]

# 查看结果
print(combined_df1)

#SCI
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/Delta/extractDIS'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df2 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df2 = combined_df2[['user_loc', 'year_week', 'W1_DistanceHIV']]

# 查看结果
print(combined_df2)

#COVID
import os
import pandas as pd

# 设置数据的目录路径
data_dir = 'D:/Data/COVID/Data/Delta/extractCOVID'  # 根据你的文件夹路径进行调整
# 获取该目录下的所有文件名
all_files = os.listdir(data_dir)

# 过滤出CSV文件
csv_files = [file for file in all_files if file.endswith('.csv')]

dataframes = []  # 用于存储每个文件的数据

for file in csv_files:
    df = pd.read_csv(os.path.join(data_dir, file))  # 使用os.path.join来合并目录路径和文件名
    year_week = file.split('_')[0] + '_' + file.split('_')[1]  # 提取年份和周数
    df['year_week'] = year_week  # 添加一列表示年份和周数
    dataframes.append(df)

# 使用concat函数将所有的DataFrame合并成一个
combined_df3 = pd.concat(dataframes, ignore_index=True)

# 如果需要，你可以重新调整列的顺序
combined_df3 = combined_df3[['fips', 'year_week', 'weekly_total_7_day_avg']]
combined_df3.rename(columns={'fips': 'user_loc'}, inplace=True)

# 查看结果
print(combined_df3)



# 将4个数据集合并为一个
merged_df = combined_df.merge(combined_df1, on=['user_loc', 'year_week'], how='outer')
merged_df = merged_df.merge(combined_df2, on=['user_loc', 'year_week'], how='outer')
merged_df = merged_df.merge(combined_df3, on=['user_loc', 'year_week'], how='outer')

# 显示合并后的数据集
print(merged_df)
merged_df.columns

#SCI
sci_df = pd.read_csv('D:/Data/COVID/Data/totalSCIeachCounty.csv')
final_df = merged_df.merge(sci_df, on='user_loc', how='outer')

#PCI
pci_df = pd.read_csv('D:/Data/COVID/Data/totalPCIeachCounty.csv')
pci_df.rename(columns={'place_i': 'user_loc'}, inplace=True)
final_df = final_df.merge(pci_df, on='user_loc', how='outer')

111111111111111
import numpy as np
final_df.columns
# 对需要的列进行对数变换，并将结果存储到新的列中
final_df['log_pcicov'] = np.log(final_df['W1_PCIHIV'])
final_df['log_scicov'] = np.log(final_df['W1_SCIHIV'])
final_df['log_discov'] = np.log(final_df['W1_DistanceHIV'])
final_df['log_incidence'] = np.log(final_df['weekly_total_7_day_avg'])
final_df['log_pci'] = np.log(final_df['pci'])
final_df['log_sci'] = np.log(final_df['scaled_sci'])

# 显示DataFrame以检查结果
print(final_df)
final_df.columns
final_df.rename(columns={'user_loc': 'fips'}, inplace=True)
final_df.rename(columns={'W1_PCIHIV': 'pcicov'}, inplace=True)
final_df.rename(columns={'W1_SCIHIV': 'scicov'}, inplace=True)
final_df.rename(columns={'W1_DistanceHIV': 'discov'}, inplace=True)

# 删除包含NaN值的行
final_df = final_df.dropna()


111111111111111#确定每个县是否有13个连续的周的数据
# 计算每个县的数据量
county_counts = final_df.groupby('fips').size()

# 找出数据量不等于13的县的列表
counties_to_remove = county_counts[county_counts != 16].index

# 只保留数据量等于13的县的数据
final_df = final_df[~final_df['fips'].isin(counties_to_remove)]


#rural-urban
urbanrural = pd.read_csv('D:/Data/COVID/Data/UrbanRural.csv')
urbanrural.rename(columns={'FIPS': 'fips'}, inplace=True)
final_df = final_df.merge(urbanrural, on='fips', how='outer')
# 删除包含NaN值的行
final_df = final_df.dropna()
final_df.to_csv('D:/Data/COVID/Data/Delta/FinaldataHIVdiagnoseUSforRNNDeltaWeighted.csv', index=False)

# 删除包含NaN值的行
final_df.columns
final_df = final_df.dropna()
final_df = final_df[final_df['weekly_total_7_day_avg'] != 0]
final_df = final_df[final_df['log_incidence'] != 0]

# 保存到 CSV 文件
final_df.to_csv('D:/Data/COVID/Data/Delta/FinaldataHIVdiagnoseUSforRNNDeltaWeighted.csv', index=False)


11111
####Results: mapping parts.
1111
# Read SCI TSV file into DataFrame
df = pd.read_table('D:/Data/COVID/Data/county_county.tsv')
df.rename(columns={'fr_loc': 'fips'}, inplace=True)


#Manhattan is New York County (ANSI / FIPS 36061)
df_selected = df.loc[df['user_loc'] == 36061]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/NewYorkCountySCI.csv', index=False)

#San Francisco, City and County (ANSI / FIPS 06075)
df_selected = df.loc[df['user_loc'] == 6075]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/SanFranCountySCI.csv', index=False)

#Richland of South Carolina, FIPS
df_selected = df.loc[df['user_loc'] == 45079]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/RichlandCountySCI.csv', index=False)


11111
#Calculate PCI
# Read PCI 2018 file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/US_County_PCI_2019.csv')
df2 = df2.drop(columns=['shared_users', 'place_i_users','place_j_users','dir_pci'])
df2.rename(columns={'place_j': 'fips'}, inplace=True)
df2.rename(columns={'place_i': 'user_loc'}, inplace=True)


#Manhattan is New York County (ANSI / FIPS 36061)
df_selected = df2.loc[df2['user_loc'] == 36061]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/NewYorkCountyPCI.csv', index=False)

#San Francisco, City and County (ANSI / FIPS 06075)
df_selected = df2.loc[df2['user_loc'] == 6075]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/SanFranCountyPCI.csv', index=False)

#Richland of South Carolina, FIPS
df_selected = df2.loc[df2['user_loc'] == 45079]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/RichlandCountyPCI.csv', index=False)


11111
#GeoDistanceWeight
# Read file into DataFrame
df2 = pd.read_csv('D:/Data/COVID/Data/Distancetextfile_part1and2.csv')

df2.rename(columns={'cb_2019_us': 'fips'}, inplace=True)
df2.rename(columns={'a': 'user_loc'}, inplace=True)


#Manhattan is New York County (ANSI / FIPS 36061)
df_selected = df2.loc[df2['user_loc'] == 36061]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/NewYorkCountyDIS.csv', index=False)

#San Francisco, City and County (ANSI / FIPS 06075)
df_selected = df2.loc[df2['user_loc'] == 6075]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/SanFranCountyDIS.csv', index=False)

#Richland of South Carolina, FIPS
df_selected = df2.loc[df2['user_loc'] == 45079]
print(df_selected)
df_selected.to_csv('D:/Data/COVID/Data/FinalResults06302023/RichlandCountyDIS.csv', index=False)



import pandas as pd
import matplotlib.pyplot as plt

# 读取CSV文件
df = pd.read_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNN.csv')
df.columns

# 预处理数据，将其转换为我们可以更容易操作的格式
data = {}
for fips in df['fips'].unique():
    data[fips] = {
        "COVID incidence": df[df['fips'] == fips].set_index('year_week')['weekly_total_7_day_avg'].to_dict(),
        "PCIWeightedCOVID": df[df['fips'] == fips].set_index('year_week')['pcicov'].to_dict(),
        "SCIWeightedCOVID": df[df['fips'] == fips].set_index('year_week')['scicov'].to_dict(),
    }

# 预处理数据，将其转换为我们可以更容易操作的格式
data = {}
for fips in df['fips'].unique():
    data[fips] = {
        "COVID incidence": df[(df['fips'] == fips) & (df['year_week'] <= '2021_8')].set_index('year_week')['weekly_total_7_day_avg'].to_dict(),
        "PCIWeightedCOVID": df[(df['fips'] == fips) & (df['year_week'] <= '2021_8')].set_index('year_week')['pcicov'].to_dict(),
        "SCIWeightedCOVID": df[(df['fips'] == fips) & (df['year_week'] <= '2021_8')].set_index('year_week')['scicov'].to_dict(),
    }

# 定义颜色，将RGB值转化为[0, 1]范围内
#colors = [(44/255, 164/255, 44/255), (255/255, 127/255, 14/255), (31/255, 119/255, 180/255)]


# 定义年份范围
year_week_COVID = list(range(2020_49, 2020_54))
year_week_rest = list(range(2020_49, 2020_54))


# Create figure with 2x2 layout
fig, ax = plt.subplots(2, 2, figsize=(15, 15))

# Define thresholds
thresholds = [500, 4000000, 10000000000, 800]

# Plot each county's HIV prevalence
for fips, metrics in data.items():
    values = np.array(list(metrics["COVID incidence"].values()))
    if np.any(values > thresholds[0]):
        continue
    ax[0,0].plot(list(metrics["COVID incidence"].keys()), values, label=fips, color='pink', linewidth=0.2)
ax[0,0].set_title("COVID incidence")
ax[0,0].axhline(thresholds[0], color='red', linewidth=0.5)  # add threshold line
ax[0,0].set_xticks(year_week_COVID)

# Plot each county's population density
for fips, metrics in data.items():
    values = np.array(list(metrics["PCIWeightedCOVID"].values()))
    if np.any(values > thresholds[1]):
        continue
    ax[0,1].plot(list(metrics["PCIWeightedCOVID"].keys()), values, label=fips, color='pink', linewidth=0.2)
ax[0,1].set_title("PCIWeightedCOVID")
ax[0,1].axhline(thresholds[1], color='red', linewidth=0.5)  # add threshold line
ax[0,1].set_xticks(year_week_rest)

# Plot each county's black population percentage
for fips, metrics in data.items():
    values = np.array(list(metrics["SCIWeightedCOVID"].values()))
    if np.any(values > thresholds[2]):
        continue
    ax[1,0].plot(list(metrics["SCIWeightedCOVID"].keys()), values, label=fips, color='pink', linewidth=0.2)
ax[1,0].set_title("SCIWeightedCOVID")
ax[1,0].axhline(thresholds[2], color='red', linewidth=0.5)  # add threshold line
ax[1,0].set_xticks(year_week_rest)

plt.tight_layout()
plt.show()



# 假设你想生成一列名为'C'的随机浮点数数据，长度和df的行数相同
df['Random'] = np.random.rand(len(df))
df.to_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataUSforRNNAddRandomColumn.csv')

# 读取CSV文件
df = pd.read_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataHIVdiagnoseUSforRNNDeltaWeighted.csv')

# 假设你想生成一列名为'C'的随机浮点数数据，长度和df的行数相同
df['Random'] = np.random.rand(len(df))
df.to_csv('D:/Data/COVID/Data/ResultsofWholeUS/FinaldataHIVdiagnoseUSforRNNDeltaWeightedAddRandomColumn.csv')















